In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import csv
import random
import re
import os
import unicodedata
import codecs
import itertools

In [2]:
CUDA = torch.cuda.is_available()
device = torch.device("cuda" if CUDA else "cpu")

## Part 1: Data Preprocessing

In [3]:
lines_filepath = os.path.join("cornell movie-dialogs corpus", "movie_lines.txt")
conv_filepath = os.path.join("cornell movie-dialogs corpus", "movie_conversations.txt")

print(lines_filepath)
print(conv_filepath)

cornell movie-dialogs corpus\movie_lines.txt
cornell movie-dialogs corpus\movie_conversations.txt


In [4]:
# Visualize some lines in file
with open(lines_filepath, 'r') as f:
    lines = f.readlines()
for line in lines[:8]:
    print(line.strip())

L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!
L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!
L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.
L984 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ She okay?
L925 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Let's go.
L924 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ Wow
L872 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Okay -- you're gonna need to learn how to lie.
L871 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ No


In [5]:
# Split each line into a dictionary of fields (lineID, characterID, movieID, character, text)
line_fields = ["lineID", "characterID", "movieID", "character", "text"]
lines = {}

with open(lines_filepath, 'r', encoding='iso-8859-1') as f:
    for line in f:
        values = line.split(' +++$+++ ')
        lineObj = {}
        for i, field in enumerate(line_fields):
            lineObj[field] = values[i]
        lines[lineObj['lineID']] = lineObj

In [6]:
conv_fields = ["character1ID", "character2ID", "movieID", "utteranceIDs"]
conversations = []

with open(conv_filepath, 'r', encoding='iso-8859-1') as f:
    for line in f:
        values = line.split(' +++$+++ ')
        convObj = {}
        for i, field in enumerate(conv_fields):
            convObj[field] = values[i]
        lineIDs = eval(convObj['utteranceIDs'])
        
        convObj["lines"] = []
        for lineID in lineIDs:
            convObj["lines"].append(lines[lineID])
        conversations.append(convObj)

In [7]:
# Extract pairs of sentences from conversations
qa_pairs = []
for conversation in conversations:
    for i in range(len(conversation["lines"]) - 1):
        inputLine = conversation["lines"][i]["text"].strip()
        targetLine = conversation["lines"][i+1]["text"].strip()
        if inputLine and targetLine:
            qa_pairs.append([inputLine, targetLine])

In [8]:
# Define path to new file
datafile = os.path.join("cornell movie-dialogs corpus", "formatted_movie_lines.txt")
delimiter = "\t"
delimiter = str(codecs.decode(delimiter, "unicode_escape"))
print("Delimiter: ", delimiter)

print("Writting newly formatted file...")
with open(datafile, "w", encoding="utf-8") as f:
    writer = csv.writer(f, delimiter=delimiter)
    for pair in qa_pairs:
        writer.writerow(pair)
print("Done writting to file")

Delimiter:  	
Writting newly formatted file...
Done writting to file


In [9]:
# Visualize some lines
datafile = os.path.join("cornell movie-dialogs corpus", "formatted_movie_lines.txt")
with open(datafile, 'rb') as f:
    lines = f.readlines()
for line in lines[:5]:
    print(line)

b"Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.\tWell, I thought we'd start with pronunciation, if that's okay with you.\r\r\n"
b"Well, I thought we'd start with pronunciation, if that's okay with you.\tNot the hacking and gagging and spitting part.  Please.\r\r\n"
b"Not the hacking and gagging and spitting part.  Please.\tOkay... then how 'bout we try out some French cuisine.  Saturday?  Night?\r\r\n"
b"You're asking me out.  That's so cute. What's your name again?\tForget it.\r\r\n"
b"No, no, it's my fault -- we didn't have a proper introduction ---\tCameron.\r\r\n"


In [10]:
PAD_token = 0 # padding short sentence
SOS_token = 1 # starting of sentence
EOS_token = 2 # ending of sentence

class Vocabulary:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3
        
    def addSentence(self, sentence):
        for word in sentence.split(" "):
            self.addWord(word)
            
    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1
            
    # remove word below a certain count threshold
    def trim(self, min_count):
        keep_words = []
        for k, v in self.word2count.items():
            if v >= min_count:
                keep_words.append(k)
        print("keep_word {}/{} = {:.4f}".format(len(keep_words), len(self.word2index), len(keep_words)/len(self.word2index)))
        # Reintialize dictionaries
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3
        
        for word in keep_words:
            self.addWord(word)   

#### Normalizing string

In [11]:
# Turn unicode string to ASCII
def unicodeToAscii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != "Mn")

In [12]:
# Test unicodeToAscii
unicodeToAscii("Montréal,François...")

'Montreal,Francois...'

In [13]:
# Lowercase, trim whitespace, lines... etc and remove non-leter character
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()
    return s

In [14]:
# Test normalizeString
normalizeString("aaa123aa!s's   dd?")

'aaa aa !s s dd ?'

In [15]:
datafile = os.path.join("cornell movie-dialogs corpus", "formatted_movie_lines.txt")

print("Reading and processing file...Please wait")
lines = open(datafile, encoding="utf-8").read().strip().split("\n")
pairs = [[normalizeString(s) for s in pair.split("\t")] for pair in lines]
print("Done reading!")

vocab = Vocabulary("cornell movie-dialogs corpus")

Reading and processing file...Please wait
Done reading!


#### Filtering 

In [16]:
MAX_LENGTH = 10
def filterPair(p):
    return len(p[0].split()) < MAX_LENGTH and len(p[1].split()) < MAX_LENGTH

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [17]:
pairs = [pair for pair in pairs if len(pair) > 1]
print("There are {} pairs/conversations in the dataset".format(len(pairs)))
pairs = filterPairs(pairs)
print("After filtering, there are {} pairs/conversations in the dataset".format(len(pairs)))

There are 221282 pairs/conversations in the dataset
After filtering, there are 64271 pairs/conversations in the dataset


In [18]:
for pair in pairs:
    vocab.addSentence(pair[0])
    vocab.addSentence(pair[1])
print("Counted words: ", vocab.num_words)
for pair in pairs[:10]:
    print(pair)

Counted words:  18008
['there .', 'where ?']
['you have my word . as a gentleman', 'you re sweet .']
['hi .', 'looks like things worked out tonight huh ?']
['you know chastity ?', 'i believe we share an art instructor']
['have fun tonight ?', 'tons']
['well no . . .', 'then that s all you had to say .']
['then that s all you had to say .', 'but']
['but', 'you always been this selfish ?']
['do you listen to this crap ?', 'what crap ?']
['what good stuff ?', 'the real you .']


In [19]:
MIN_COUNT = 3 # threshold

def trimRareWords(vocab, pairs, MIN_COUNT):
    vocab.trim(MIN_COUNT)
    
    keep_pairs = []
    for pair in pairs:
        input_sentence = pair[0]
        output_sentence = pair[1]
        keep_input = True
        keep_output = True
        
        for word in input_sentence.split(" "):
            if word not in vocab.word2index:
                keep_input = False
                break
        for word in output_sentence.split(" "):
            if word not in vocab.word2index:
                keep_output = False
                break
        
        if keep_input and keep_output:
            keep_pairs.append(pair)
            
    print("Trimmed from {} pairs to {}, {:.4f} of total".format(len(pairs), len(keep_pairs), len(keep_pairs)/len(pairs)))
    return keep_pairs

In [20]:
pairs = trimRareWords(vocab, pairs, MIN_COUNT)

keep_word 7823/18005 = 0.4345
Trimmed from 64271 pairs to 53165, 0.8272 of total


## Part 2: Data preparation

In [21]:
def indexesFromSentence(vocab, sentence):
    return [vocab.word2index[word] for word in sentence.split(" ")] + [EOS_token]

In [22]:
# Test indexesFromSentence
print(pairs[1][0])
print(indexesFromSentence(vocab, pairs[1][0]))

you have my word . as a gentleman
[7, 8, 9, 10, 4, 11, 12, 13, 2]


In [23]:
inp = []
out = []
i = 0
for pair in pairs[:15]:
    inp.append(pair[0])
    out.append(pair[1])
indexes = [indexesFromSentence(vocab, sentence) for sentence in inp]
print(inp)
print(indexes)

['there .', 'you have my word . as a gentleman', 'hi .', 'have fun tonight ?', 'well no . . .', 'then that s all you had to say .', 'but', 'do you listen to this crap ?', 'what good stuff ?', 'wow', 'she okay ?', 'they do to !', 'did you change your hair ?', 'no .', 'who ?']
[[3, 4, 2], [7, 8, 9, 10, 4, 11, 12, 13, 2], [16, 4, 2], [8, 31, 22, 6, 2], [33, 34, 4, 4, 4, 2], [35, 36, 37, 38, 7, 39, 40, 41, 4, 2], [42, 2], [47, 7, 48, 40, 45, 49, 6, 2], [50, 51, 52, 6, 2], [58, 2], [61, 62, 6, 2], [65, 47, 40, 66, 2], [68, 7, 69, 70, 71, 6, 2], [34, 4, 2], [77, 6, 2]]


In [24]:
def zeroPadding(l, fillvalue=0):
    return list(itertools.zip_longest(*l, fillvalue=fillvalue))

In [25]:
leng = [len(ind) for ind in indexes]
max(leng)

10

In [26]:
leng

[3, 9, 3, 5, 6, 10, 2, 8, 5, 2, 4, 5, 7, 3, 3]

In [27]:
test_result = zeroPadding(indexes)
print(len(test_result))
test_result

10


[(3, 7, 16, 8, 33, 35, 42, 47, 50, 58, 61, 65, 68, 34, 77),
 (4, 8, 4, 31, 34, 36, 2, 7, 51, 2, 62, 47, 7, 4, 6),
 (2, 9, 2, 22, 4, 37, 0, 48, 52, 0, 6, 40, 69, 2, 2),
 (0, 10, 0, 6, 4, 38, 0, 40, 6, 0, 2, 66, 70, 0, 0),
 (0, 4, 0, 2, 4, 7, 0, 45, 2, 0, 0, 2, 71, 0, 0),
 (0, 11, 0, 0, 2, 39, 0, 49, 0, 0, 0, 0, 6, 0, 0),
 (0, 12, 0, 0, 0, 40, 0, 6, 0, 0, 0, 0, 2, 0, 0),
 (0, 13, 0, 0, 0, 41, 0, 2, 0, 0, 0, 0, 0, 0, 0),
 (0, 2, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0),
 (0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0)]

In [28]:
def binaryMatrix(l, value=0):
    m = []
    for i, seq in enumerate(l):
        m.append([])
        for token in seq:
            if token == PAD_token:
                m[i].append(0)
            else:
                m[i].append(1)
    return m

In [29]:
binary_result = binaryMatrix(test_result)
binary_result

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1],
 [0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0],
 [0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0],
 [0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0],
 [0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0],
 [0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [30]:
def inputVar(l, vocab):
    indexes_batch = [indexesFromSentence(vocab, sentence) for sentence in l]
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    padVar = torch.LongTensor(padList)
    return padVar, lengths

In [31]:
def outputVar(l, vocab):
    indexes_batch = [indexesFromSentence(vocab, sentence) for sentence in l]
    max_target_len = max([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    mask = binaryMatrix(padList)
    mask = torch.ByteTensor(mask)
    padVar = torch.LongTensor(padList)
    return padVar, mask, max_target_len

In [32]:
def batch2TrainData(vocab, pair_batch):
    pair_batch.sort(key=lambda x: len(x[0].split(" ")), reverse=True)
    input_batch, output_batch = [], []
    for pair in pair_batch:
        input_batch.append(pair[0])
        output_batch.append(pair[1])
    inp, lengths = inputVar(input_batch, vocab)
    out, mask, max_target_len = outputVar(output_batch, vocab)
    return inp, lengths, out, mask, max_target_len

In [33]:
small_batch_size = 5
batches = batch2TrainData(vocab, [random.choice(pairs) for _ in range(small_batch_size)])

input_variable, lengths, output_variable, mask, max_target_len = batches

print("input_variable")
print(input_variable)
print("lengths")
print(lengths)
print("output_varialbe")
print(output_variable)
print("mask")
print(mask)
print("max_target_len")
print(max_target_len)

input_variable
tensor([[ 654,    7, 6763,   25,   50],
        [  25,  250, 6762,  200,   37],
        [ 247,   44, 1766,  483,  101],
        [ 117,   96,   40,  572,  206],
        [  24,  180,  359,   25,   75],
        [   7,   56,    7,  102,    6],
        [ 102,  960,    4,  273,    2],
        [ 467,   19,    2,    2,    0],
        [   4,    4,    0,    0,    0],
        [   2,    2,    0,    0,    0]])
lengths
tensor([10, 10,  8,  8,  7])
output_varialbe
tensor([[ 192,   25,   77,  112,   59],
        [   4,  425,    6,  197,   83],
        [ 534,    7,    2,  117,   60],
        [  12,    8,    0,    4,   66],
        [1230,    4,    0,    2,    2],
        [1382,    4,    0,    0,    0],
        [ 154,    4,    0,    0,    0],
        [ 164,    2,    0,    0,    0],
        [   4,    0,    0,    0,    0],
        [   2,    0,    0,    0,    0]])
mask
tensor([[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 0, 1, 1],
        [1, 1, 0, 1, 1],


## Part 3: Building the model

In [34]:
class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, embedding, n_layers=1, dropout=0):
        super(EncoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding = embedding
        
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers==1 else dropout), bidirectional=True)
        
    def forward(self, input_seq, input_lengths, hidden=None):
        embedded = self.embedding(input_seq)
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
        outputs, hidden = self.gru(packed, hidden)
        outputs, _ = torch.nn.utils.rnn.pad_packed_sequence(outputs)
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, :, self.hidden_size:]
        return output, hidden

In [35]:
class Attn(nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        self.method = method
        self.hidden_size = hidden_size
        
    def dot_score(self, hidden, encoder_output):
        return torch.sum(hidden * encoder_output, dim=2)
    
    def foward(self, hidden, encoder_outputs):
        attn_energies = self.dot_score(hidden, encoder_outputs)
        # tranpose
        attn_energies = attn_energies.t()
        return F.softmax(attn_energies, dim=1).unsqueeze(1)

In [36]:
class LuongAttnDecoderRNN(nn.Module):
    def __init__(self, attn_model, embedding, hidden_size, output_size, n_layers=1, dropout=0.1):
        super(LuongAttnDecoderRNN, self).__init__()
        self.attn_model = attn_model
        self.embedding = embedding
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout
        
        self.embedding_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers==1 else dropout))
        self.concat = nn.Linear(hidden_size*2, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        
        self.attn = Attn(attn_model, hidden_size)
        
    def forward(self, input_step, last_hidden, encoder_outputs):
        embedded = self.embedding(input_step)
        embedded = self.embedding_dropout(embedded)
        rnn_output, hidden = self.gru(embedded, last_hidden)
        attn_weights = self.attn(rnn_output, encoder_outputs)
        context = attn_weights.bmm(encoder_outputs.tranpose(0, 1))
        rnn_output = rnn_output.squeeze(0)
        context = context.squeeze(1)
        concat_input = torch.cat((rnn_output, context), 1)
        concat_output = torch.tanh(self.concat(concat_input))
        output = self.out(concat_output)
        output = F.softmax(output, dim=1)
        return output, hidden

## Part 4: Training the model

In [37]:
def maskNLLLoss(decoder_out, target, mask):
    nTotal = mask.sum()
    target = target.view(-1, 1)
    # decoder_output shape (batch_size, vocab_size), target shape (batch_size, 1)
    gathered_tensor = torch.gather(decoder_out, 1, target)
    crossEntropy = -torch.log(gathered_tensor)
    loss = crossEntropy.masked_select(mask)
    loss = loss.mean()
    loss = loss.to(device)
    return loss, nTotal.item()

In [ ]:
# Visualize what's happening in the loss function
dec_o = torch.rand(5, 7)
dec_o = F.softmax(dec_o, dim=1)
tar = torch.tensor([2, 1, 5, 4, 0], dtype=torch.long)
tar = tar.view(-1, 1)
mask = torch.tensor([1, 0, 1, 1, 0], dtype=torch.uint8)
print(dec_o)
print(tar)
gath_ten = torch.gather(dec_o, 1, tar)
print(gath_ten)
print(gath_ten.shape)
crossEntropy = -torch.log(gath_ten)
